In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
#importing the PCA scaling library
from sklearn.decomposition import PCA
#importing the feature scaling library
from sklearn.preprocessing import StandardScaler
#Import Sklearn package's data splitting function which is based on random function
from sklearn.model_selection import train_test_split
# Import Linear Regression, Ridge and Lasso machine learning library
from sklearn.linear_model import LogisticRegression
# Import KNN Regressor machine learning library
from sklearn.tree import DecisionTreeClassifier
# Import ensemble machine learning library
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor)
# Import support vector regressor machine learning library
from sklearn.svm import SVC
#Import the metrics
from sklearn import metrics
#Import the Voting regressor for Ensemble
from sklearn.ensemble import VotingRegressor
# Import stats from scipy
from scipy import stats
# Import zscore for scaling
from scipy.stats import zscore
#importing the metrics
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
#importing the K fold
from sklearn.model_selection import KFold
#importing the cross validation score
from sklearn.model_selection import cross_val_score
#importing the preprocessing library
from sklearn import preprocessing
# importing the Polynomial features
from sklearn.preprocessing import PolynomialFeatures
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
loans_main = pd.read_csv('loans_main.csv')
loans_main.head()

,Unnamed: 0,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,...,tax_liens,loan_status,policy_code,FICO_mean,count,is_amnt_outlier,amount_bins,FICO_group,is_income_outlier,credit_history
0,0,5000.0,5000.0,4975.0,36,10.65,162.87,B,6,10,...,0.0,Fully Paid,1,737.0,1,0,"(3000.0, 5000.0]",Good,0,9830.0
1,1,2500.0,2500.0,2500.0,60,15.27,59.83,C,13,0,...,0.0,Charged Off,1,742.0,1,0,"(499.999, 3000.0]",Very Good,0,4627.0
2,2,2400.0,2400.0,2400.0,36,15.96,84.33,C,14,10,...,0.0,Fully Paid,1,737.0,1,0,"(499.999, 3000.0]",Good,0,3682.0
3,3,10000.0,10000.0,10000.0,36,13.49,339.31,C,10,10,...,0.0,Fully Paid,1,692.0,1,0,"(9600.0, 11000.0]",Good,0,5782.0
4,4,3000.0,3000.0,3000.0,60,12.69,67.79,B,9,1,...,0.0,Fully Paid,1,697.0,1,0,"(499.999, 3000.0]",Good,0,5813.0


In [3]:
loans_main.drop('Unnamed: 0', axis = 1, inplace = True)

# Feature Eng

I've already created 7 new features: policy_code, FICO_mean, count, amnt_outliers, is_amnt_outlier, is_income_outlier, credit_history. Now I'm going to change the categorical features to binary ones. I'm staring from changing the loan_status. I also change the 'term' feature to binary as it actualy reprezent two option borrowers get.

In [4]:
loans_main.loan_status.replace(["Fully Paid", "Charged Off"], [1, 0], inplace = True)

In [5]:
loans_main.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,tax_liens,loan_status,policy_code,FICO_mean,count,is_amnt_outlier,amount_bins,FICO_group,is_income_outlier,credit_history
0,5000.0,5000.0,4975.0,36,10.65,162.87,B,6,10,RENT,...,0.0,1,1,737.0,1,0,"(3000.0, 5000.0]",Good,0,9830.0
1,2500.0,2500.0,2500.0,60,15.27,59.83,C,13,0,RENT,...,0.0,0,1,742.0,1,0,"(499.999, 3000.0]",Very Good,0,4627.0
2,2400.0,2400.0,2400.0,36,15.96,84.33,C,14,10,RENT,...,0.0,1,1,737.0,1,0,"(499.999, 3000.0]",Good,0,3682.0
3,10000.0,10000.0,10000.0,36,13.49,339.31,C,10,10,RENT,...,0.0,1,1,692.0,1,0,"(9600.0, 11000.0]",Good,0,5782.0
4,3000.0,3000.0,3000.0,60,12.69,67.79,B,9,1,RENT,...,0.0,1,1,697.0,1,0,"(499.999, 3000.0]",Good,0,5813.0


In [6]:
loans_main.term.replace([36,60], [0, 1], inplace = True)

In [7]:
# now I'm going to change date formats into intgers
loans_main['issue_d']= pd.to_datetime(loans_main['issue_d']).apply(lambda x: int(x.strftime('%Y')))

In [8]:
loans_main['earliest_cr_line']= pd.to_datetime(loans_main['earliest_cr_line']).apply(lambda x: int(x.strftime('%Y')))
loans_main['last_credit_pull_d']= pd.to_datetime(loans_main['last_credit_pull_d']).apply(lambda x: int(x.strftime('%Y')))
#loans_main['last_pymnt_d']= pd.to_datetime(loans_main['last_pymnt_d']).apply(lambda x: int(x.strftime('%Y')))

# I've decided to remove the highest amounts as they disturb the analysis (outliers) and replace them with median
loans_main.annual_inc = np.where(loans_main["annual_inc"] > 2620000, float(loans_main["annual_inc"].median()), loans_main["annual_inc"])
loans_main.annual_inc.describe()

In [9]:
# replacing bins with numerical values
bins = np.array(loans_main.amount_bins.unique())
bins.sort()
print(bins)

['(11000.0, 13200.0]' '(13200.0, 16000.0]' '(16000.0, 20400.0]'
 '(20400.0, 35000.0]' '(3000.0, 5000.0]' '(499.999, 3000.0]'
 '(5000.0, 6000.0]' '(6000.0, 7500.0]' '(7500.0, 9600.0]'
 '(9600.0, 11000.0]']


In [10]:
loans_main.amount_bins.replace(['(499.999, 3000.0]', '(3000.0, 5000.0]','(5000.0, 6000.0]', '(6000.0, 7500.0]', '(7500.0, 9600.0]', '(9600.0, 11000.0]','(11000.0, 13200.0]', '(13200.0, 16000.0]', '(16000.0, 20400.0]', '(20400.0, 35000.0]'], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], inplace = True)
loans_main.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,tax_liens,loan_status,policy_code,FICO_mean,count,is_amnt_outlier,amount_bins,FICO_group,is_income_outlier,credit_history
0,5000.0,5000.0,4975.0,0,10.65,162.87,B,6,10,RENT,...,0.0,1,1,737.0,1,0,2,Good,0,9830.0
1,2500.0,2500.0,2500.0,1,15.27,59.83,C,13,0,RENT,...,0.0,0,1,742.0,1,0,1,Very Good,0,4627.0
2,2400.0,2400.0,2400.0,0,15.96,84.33,C,14,10,RENT,...,0.0,1,1,737.0,1,0,1,Good,0,3682.0
3,10000.0,10000.0,10000.0,0,13.49,339.31,C,10,10,RENT,...,0.0,1,1,692.0,1,0,6,Good,0,5782.0
4,3000.0,3000.0,3000.0,1,12.69,67.79,B,9,1,RENT,...,0.0,1,1,697.0,1,0,1,Good,0,5813.0


In [11]:
# replacing grade with numerical values
grade = np.array(loans_main.grade.unique())
grade.sort()
print(grade)

['A' 'B' 'C' 'D' 'E' 'F' 'G']


In [12]:
loans_main.grade.replace(grade, [1, 2, 3, 4, 5, 6, 7], inplace = True)

In [13]:
loans_main.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,tax_liens,loan_status,policy_code,FICO_mean,count,is_amnt_outlier,amount_bins,FICO_group,is_income_outlier,credit_history
0,5000.0,5000.0,4975.0,0,10.65,162.87,2,6,10,RENT,...,0.0,1,1,737.0,1,0,2,Good,0,9830.0
1,2500.0,2500.0,2500.0,1,15.27,59.83,3,13,0,RENT,...,0.0,0,1,742.0,1,0,1,Very Good,0,4627.0
2,2400.0,2400.0,2400.0,0,15.96,84.33,3,14,10,RENT,...,0.0,1,1,737.0,1,0,1,Good,0,3682.0
3,10000.0,10000.0,10000.0,0,13.49,339.31,3,10,10,RENT,...,0.0,1,1,692.0,1,0,6,Good,0,5782.0
4,3000.0,3000.0,3000.0,1,12.69,67.79,2,9,1,RENT,...,0.0,1,1,697.0,1,0,1,Good,0,5813.0


In [14]:
# I change the cateegorical values to binary ones creating additional features
loans_dummies = pd.get_dummies(loans_main.drop(['loan_status', 'addr_state', 'mths_since_last_delinq', 'mths_since_last_record', 'last_pymnt_d', 'count', 'funded_amnt_inv', 'fico_range_low', 'fico_range_high', 'total_pymnt_inv', 'last_fico_range_high', 'last_fico_range_low'], axis = 1))
loans_dummies.head()

,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,issue_d,...,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,FICO_group_Exeptional,FICO_group_Fair,FICO_group_Good,FICO_group_Very Good
0,5000.0,5000.0,0,10.65,162.87,2,6,10,24000.0,2011,...,0,0,0,0,0,0,0,0,1,0
1,2500.0,2500.0,1,15.27,59.83,3,13,0,30000.0,2011,...,0,0,0,0,0,0,0,0,0,1
2,2400.0,2400.0,0,15.96,84.33,3,14,10,12252.0,2011,...,0,0,0,1,0,0,0,0,1,0
3,10000.0,10000.0,0,13.49,339.31,3,10,10,49200.0,2011,...,0,1,0,0,0,0,0,0,1,0
4,3000.0,3000.0,1,12.69,67.79,2,9,1,80000.0,2011,...,0,1,0,0,0,0,0,0,1,0


In [15]:
dummies_loan_status = pd.get_dummies(loans_main.drop(['addr_state', 'mths_since_last_delinq', 'mths_since_last_record', 'last_pymnt_d', 'count', 'funded_amnt_inv', 'fico_range_low', 'fico_range_high', 'total_pymnt_inv', 'last_fico_range_high', 'last_fico_range_low'], axis = 1))
dummies_loan_status.to_csv('dummies_loan_status.csv')

In [16]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_row', None)
loans_dummies.dtypes

loan_amnt                              float64
funded_amnt                            float64
term                                     int64
int_rate                               float64
installment                            float64
grade                                    int64
sub_grade                                int64
emp_length                               int64
annual_inc                             float64
issue_d                                  int64
dti                                    float64
delinq_2yrs                            float64
earliest_cr_line                         int64
inq_last_6mths                         float64
open_acc                               float64
pub_rec                                float64
revol_bal                              float64
revol_util                             float64
total_acc                              float64
total_pymnt                            float64
total_rec_prncp                        float64
total_rec_int

In [17]:
# checking the skewness
loans_dummies.skew()

loan_amnt                                1.064970
funded_amnt                              1.085767
term                                     1.102458
int_rate                                 0.238810
installment                              1.125240
grade                                    0.755431
sub_grade                                0.769807
emp_length                               0.245659
annual_inc                              29.031679
issue_d                                 -1.226890
dti                                     -0.029922
delinq_2yrs                              5.435314
earliest_cr_line                        -1.156277
inq_last_6mths                           3.454130
open_acc                                 1.035940
pub_rec                                  4.607298
revol_bal                               11.012291
revol_util                              -0.043150
total_acc                                0.818775
total_pymnt                              1.379195


In [18]:
X = loans_dummies
y = loans_main.loan_status

In [19]:
X_train, X_test, y_train, y_test = train_test_split(loans_dummies, y, test_size=0.2, random_state=0)

In [20]:
X_train.shape, X_test.shape

((34028, 63), (8507, 63))

In [21]:
# some features are very skew, I'll normalize them
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train = pd.DataFrame(X_train, columns= X.columns)
X_test = pd.DataFrame(X_test, columns = X.columns)

In [22]:
X_train.describe()

,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,issue_d,...,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,FICO_group_Exeptional,FICO_group_Fair,FICO_group_Good,FICO_group_Very Good
count,3.402800e+04,3.402800e+04,3.402800e+04,3.402800e+04,3.402800e+04,3.402800e+04,3.402800e+04,3.402800e+04,3.402800e+04,3.402800e+04,...,3.402800e+04,3.402800e+04,3.402800e+04,3.402800e+04,3.402800e+04,3.402800e+04,3.402800e+04,3.402800e+04,3.402800e+04,3.402800e+04
mean,-1.976129e-16,1.698932e-16,5.640512e-17,-5.208020e-16,-3.216475e-16,4.236518e-16,-1.800311e-16,-1.232663e-16,-1.866739e-16,-9.756684e-14,...,-1.069704e-15,-6.072915e-16,1.549027e-15,1.876689e-15,1.496342e-15,-1.678826e-16,-9.218451e-16,-9.805937e-16,-3.094386e-16,3.870054e-16
std,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,...,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00
min,-1.427581e+00,-1.443574e+00,-5.921026e-01,-1.819538e+00,-1.469541e+00,-1.163722e+00,-1.473141e+00,-1.328070e+00,-1.193229e+00,-3.344477e+00,...,-1.224897e-01,-3.402136e-01,-5.091965e-02,-2.215429e-01,-9.635197e-02,-1.545678e-01,-1.204971e-01,-3.167080e-01,-1.376774e+00,-5.617831e-01
25%,-8.054385e-01,-8.123961e-01,-5.921026e-01,-6.836952e-01,-7.521887e-01,-4.676814e-01,-7.653359e-01,-7.736876e-01,-5.005404e-01,-2.409065e-01,...,-1.224897e-01,-3.402136e-01,-5.091965e-02,-2.215429e-01,-9.635197e-02,-1.545678e-01,-1.204971e-01,-3.167080e-01,-1.376774e+00,-5.617831e-01
50%,-1.968213e-01,-1.812178e-01,-5.921026e-01,-4.697587e-02,-2.152698e-01,-4.676814e-01,-1.990916e-01,-2.193057e-01,-1.839124e-01,7.936172e-01,...,-1.224897e-01,-3.402136e-01,-5.091965e-02,-2.215429e-01,-9.635197e-02,-1.545678e-01,-1.204971e-01,-3.167080e-01,7.263355e-01,-5.617831e-01
75%,5.335194e-01,5.902223e-01,1.688897e+00,6.895681e-01,5.036321e-01,9.243991e-01,6.502749e-01,1.166649e+00,2.320774e-01,7.936172e-01,...,-1.224897e-01,-3.402136e-01,-5.091965e-02,-2.215429e-01,-9.635197e-02,-1.545678e-01,-1.204971e-01,-3.167080e-01,7.263355e-01,-5.617831e-01
max,3.238485e+00,3.395459e+00,1.688897e+00,3.352458e+00,4.724341e+00,3.012520e+00,3.339936e+00,1.443840e+00,6.634391e+01,7.936172e-01,...,8.163951e+00,2.939330e+00,1.963878e+01,4.513798e+00,1.037862e+01,6.469654e+00,8.298957e+00,3.157483e+00,7.263355e-01,1.780046e+00


# Modeling

In [23]:
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, recall_score, precision_score, f1_score,  roc_auc_score

In [24]:
model_svc=SVC() 
model_svc.fit(X_train,y_train)
model_svc.score(X_test,y_test)

0.9740213941459974

In [25]:
prediction_svc = model_svc.predict(X_test)
# performance on train data
print('Performance on training data using SVC:', model_svc.score(X_train,y_train))
# performance on test data
print('Performance on testing data using SVC:', model_svc.score(X_test,y_test))
#Evaluate the model using accuracy
acc_SVC = accuracy_score(y_test, prediction_svc)
#Evaluate the model using recall
recall_SVC = recall_score(y_test, prediction_svc)
#Evaluate the model using precision
precision_SVC = precision_score(y_test, prediction_svc)
#Evaluate the model using F1 score
f1_SVC = f1_score(y_test, prediction_svc)
#Evaluate the model using AUROC
AUROC_SVC = roc_auc_score(y_test, prediction_svc)

Performance on training data using SVC: 0.9806042082990478
Performance on testing data using SVC: 0.9740213941459974


In [26]:
confusion_matrix(y_test, prediction_svc)

array([[1109,  221],
       [   0, 7177]], dtype=int64)

In [27]:
results = pd.DataFrame({'Method':['SVM'], 'accuracy': acc_SVC, 'recall': recall_SVC, 'precision': precision_SVC, 'f1 score': f1_SVC, 'AUROC': AUROC_SVC},index={'1'})
results = results[['Method', 'accuracy', 'recall', 'precision', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precision,f1 score,AUROC
1,SVM,0.974021,1.0,0.970127,0.984837,0.916917


In [28]:
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True, random_state = 77)
results1 = cross_val_score(model_svc, X, y, cv=kfold)
acc_KF_SVC = np.mean(abs(results1))
AUROC_KF_SVC = np.mean(abs(cross_val_score(model_svc, X, y, scoring='roc_auc', cv=10)))
#recall_KF_SVC = np.mean(abs(cross_val_score(model_svc, X, y, scoring='recall', cv=10)))
#precision_KF_SVC = np.mean(abs(cross_val_score(model_svc, X, y, scoring='precision', cv=10)))
#f1_KF_SVC = np.mean(abs(cross_val_score(model_svc, X, y, scoring='f1', cv=10)))

In [29]:
tempResultsDf = pd.DataFrame({'Method':['SVC K Fold'], 'accuracy': [acc_KF_SVC], 'AUROC': AUROC_KF_SVC}, index={'2'}) #'recall': recall_KF_SVC, 'precision': precision_KF_SVC, 'f1 score': f1_KF_SVC,
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precision', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precision,f1 score,AUROC
1,SVM,0.974021,1.0,0.970127,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324


In [30]:
model_LR = LogisticRegression(max_iter=10000)
model_LR.fit(X_train, y_train)
model_LR.score(X_test,y_test)

0.9928294345832843

In [31]:
prediction_LR = model_LR.predict(X_test)
print('Performance on training data using Logistic Regression:',model_LR.score(X_train,y_train))
print('Performance on testing data using Logistic Regression:',model_LR.score(X_test,y_test))
acc_LR = accuracy_score(y_test, prediction_LR)
recall_LR = recall_score(y_test, prediction_LR)
precision_LR = precision_score(y_test, prediction_LR)
f1_LR = f1_score(y_test, prediction_LR)
AUROC_LR = roc_auc_score(y_test, prediction_LR)

Performance on training data using Logistic Regression: 0.9938286117315153
Performance on testing data using Logistic Regression: 0.9928294345832843


In [32]:
confusion_matrix(y_test, prediction_LR)

array([[1269,   61],
       [   0, 7177]], dtype=int64)

In [33]:
tempResultsDf = pd.DataFrame({'Method':['Logistic Regression'], 'accuracy': acc_LR, 'recall': recall_LR, 'precision': precision_LR, 'f1 score': f1_LR, 'AUROC': AUROC_LR},index={'3'})
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precision', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precision,f1 score,AUROC
1,SVM,0.974021,1.0,0.970127,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324
3,Logistic Regression,0.992829,1.0,0.991572,0.995768,0.977068


In [34]:
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True, random_state = 77)
resultsLR = cross_val_score(model_LR,X, y, cv=kfold)
acc_KF_LR=np.mean(abs(resultsLR))
AUROC_KF_LR = np.mean(abs(cross_val_score(model_LR, X, y, scoring='roc_auc', cv=10)))
#recall_KF_LR = np.mean(abs(cross_val_score(model_LR, X, y, scoring='recall', cv=10)))
#precision_KF_LR = np.mean(abs(cross_val_score(model_LR, X, y, scoring='precision', cv=10)))
#f1_KF_LR = np.mean(abs(cross_val_score(model_LR, X, y, scoring='f1', cv=10)))

In [35]:
tempResultsDf = pd.DataFrame({'Method':['Logistic Regression KFold'], 'accuracy': [acc_KF_LR], 'AUROC': AUROC_KF_LR}, index={'4'}) #'recall': recall_KF_LR, 'precision': precision_KF_LR, 'f1 score': f1_KF_LR,
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precision', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precision,f1 score,AUROC
1,SVM,0.974021,1.0,0.970127,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324
3,Logistic Regression,0.992829,1.0,0.991572,0.995768,0.977068
4,Logistic Regression KFold,0.999976,NaN,NaN,NaN,1.000000


In [36]:
model_AB = AdaBoostClassifier(random_state=1)
model_AB.fit(X_train, y_train)
model_AB.score(X_test,y_test)

0.9992946984836018

In [37]:
prediction_AB = model_AB.predict(X_test)
print('Performance on training data using AdaBoosting Classifier:',model_AB.score(X_train,y_train))
print('Performance on testing data using AdaBoosting:',model_AB.score(X_test,y_test))
acc_AB = accuracy_score(y_test, prediction_AB)
recall_AB = recall_score(y_test, prediction_AB)
precision_AB = precision_score(y_test, prediction_AB)
f1_AB = f1_score(y_test, prediction_AB)
AUROC_AB = roc_auc_score(y_test, prediction_AB)

Performance on training data using AdaBoosting Classifier: 0.9996179616786176
Performance on testing data using AdaBoosting: 0.9992946984836018


In [38]:
confusion_matrix(y_test, prediction_AB)

array([[1324,    6],
       [   0, 7177]], dtype=int64)

In [39]:
tempResultsDf = pd.DataFrame({'Method':['AdaBoost Cassifier'], 'accuracy': [acc_AB], 'recall': recall_AB, 'precision': precision_AB, 'f1 score': f1_AB, 'AUROC': AUROC_AB}, index={'5'})
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precision', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precision,f1 score,AUROC
1,SVM,0.974021,1.0,0.970127,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324
3,Logistic Regression,0.992829,1.0,0.991572,0.995768,0.977068
4,Logistic Regression KFold,0.999976,NaN,NaN,NaN,1.000000
5,AdaBoost Cassifier,0.999295,1.0,0.999165,0.999582,0.997744


In [40]:
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True, random_state = 77)
resultsAB = cross_val_score(model_AB,X, y, cv=kfold)
acc_KF_AB =np.mean(abs(resultsAB))
AUROC_KF_AB = np.mean(abs(cross_val_score(model_AB, X, y, scoring='roc_auc', cv=10)))
recall_KF_AB = np.mean(abs(cross_val_score(model_AB, X, y, scoring='recall', cv=10)))
precision_KF_AB = np.mean(abs(cross_val_score(model_AB, X, y, scoring='precision', cv=10)))
f1_KF_AB = np.mean(abs(cross_val_score(model_AB, X, y, scoring='f1', cv=10)))

In [41]:
tempResultsDf = pd.DataFrame({'Method':['AdaBoost KFold'], 'accuracy': [acc_KF_AB], 'AUROC': AUROC_KF_AB, 'recall': recall_KF_AB, 'precision': precision_KF_AB, 'f1 score': f1_KF_AB}, index={'6'})
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precision', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precision,f1 score,AUROC
1,SVM,0.974021,1.000000,0.970127,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324
3,Logistic Regression,0.992829,1.000000,0.991572,0.995768,0.977068
4,Logistic Regression KFold,0.999976,NaN,NaN,NaN,1.000000
5,AdaBoost Cassifier,0.999295,1.000000,0.999165,0.999582,0.997744
6,AdaBoost KFold,0.999248,0.940083,0.999170,0.956812,0.997667


In [42]:
model_GB= GradientBoostingClassifier(learning_rate=0.01, random_state=1)
model_GB.fit(X_train, y_train)
model_GB.score(X_test,y_test)

0.9989420477254026

In [43]:
prediction_GB = model_GB.predict(X_test)
print('Performance on training data using Gradient Boosting Classifier:',model_GB.score(X_train,y_train))
print('Performance on testing data using Gradient Boosting Classifier:',model_GB.score(X_test,y_test))
acc_GB = accuracy_score(y_test, prediction_GB)
recall_GB = recall_score(y_test, prediction_GB)
precision_GB = precision_score(y_test, prediction_GB)
f1_GB = f1_score(y_test, prediction_GB)
AUROC_GB = roc_auc_score(y_test, prediction_GB)
print('Accuracy Gradient Boosting Classifier: ', acc_GB)
print('Recall Gradient Boosting Classifier: ', recall_GB)
print('Precission Gradient Boosting Classifier: ', precision_GB)
print('F1 score Gradient Boosting Classifier: ', f1_GB)
print('AUROC Gradient Boosting Classifier: ', AUROC_GB)
print('MSE: ',metrics.mean_squared_error(y_test, prediction_GB))

Performance on training data using Gradient Boosting Classifier: 0.9987951099094863
Performance on testing data using Gradient Boosting Classifier: 0.9989420477254026
Accuracy Gradient Boosting Classifier:  0.9989420477254026
Recall Gradient Boosting Classifier:  1.0
Precission Gradient Boosting Classifier:  0.9987475647091567
F1 score Gradient Boosting Classifier:  0.9993733899603148
AUROC Gradient Boosting Classifier:  0.9966165413533834
MSE:  0.0010579522745973904


In [44]:
confusion_matrix(y_test, prediction_GB)

array([[1321,    9],
       [   0, 7177]], dtype=int64)

In [45]:
tempResultsDf = pd.DataFrame({'Method':['Gradient Boosting Classifier'], 'accuracy': [acc_GB], 'recall': recall_GB, 'precision': precision_GB, 'f1 score': f1_GB, 'AUROC': AUROC_GB}, index={'7'})
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precision', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precision,f1 score,AUROC
1,SVM,0.974021,1.000000,0.970127,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324
3,Logistic Regression,0.992829,1.000000,0.991572,0.995768,0.977068
4,Logistic Regression KFold,0.999976,NaN,NaN,NaN,1.000000
5,AdaBoost Cassifier,0.999295,1.000000,0.999165,0.999582,0.997744
6,AdaBoost KFold,0.999248,0.940083,0.999170,0.956812,0.997667
7,Gradient Boosting Classifier,0.998942,1.000000,0.998748,0.999373,0.996617


In [46]:
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True, random_state = 77)
resultsGB = cross_val_score(model_GB,X, y, cv=kfold)
acc_KF_GB =np.mean(abs(resultsGB))
AUROC_KF_GB = np.mean(abs(cross_val_score(model_GB, X, y, scoring='roc_auc', cv=10)))
recall_KF_GB = np.mean(abs(cross_val_score(model_GB, X, y, scoring='recall', cv=10)))
precision_KF_GB = np.mean(abs(cross_val_score(model_GB, X, y, scoring='precision', cv=10)))
f1_KF_GB = np.mean(abs(cross_val_score(model_GB, X, y, scoring='f1', cv=10)))

In [47]:
tempResultsDf = pd.DataFrame({'Method':['Gradient Boosting KFold'], 'accuracy': [acc_KF_GB], 'recall': recall_KF_GB, 'precision': precision_KF_GB, 'f1 score': f1_KF_GB, 'AUROC': AUROC_KF_GB}, index={'8'})
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precision', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precision,f1 score,AUROC
1,SVM,0.974021,1.000000,0.970127,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324
3,Logistic Regression,0.992829,1.000000,0.991572,0.995768,0.977068
4,Logistic Regression KFold,0.999976,NaN,NaN,NaN,1.000000
5,AdaBoost Cassifier,0.999295,1.000000,0.999165,0.999582,0.997744
6,AdaBoost KFold,0.999248,0.940083,0.999170,0.956812,0.997667
7,Gradient Boosting Classifier,0.998942,1.000000,0.998748,0.999373,0.996617
8,Gradient Boosting KFold,0.998801,0.935125,0.998783,0.951380,0.934470


In [48]:
model_XGB = xgb.XGBClassifier(random_state=1,learning_rate=0.01, early_stopping_rounds=10)
model_XGB.fit(X_train, y_train)
model_XGB.score(X_test,y_test)

0.9989420477254026

In [49]:
prediction_XGB = model_XGB.predict(X_test)
print('Performance on training data using XGB Classifier:',model_XGB.score(X_train,y_train))
print('Performance on testing data using XGB Classifier:',model_XGB.score(X_test,y_test))
acc_XGB = accuracy_score(y_test, prediction_XGB)
recall_XGB = recall_score(y_test, prediction_XGB)
precision_XGB = precision_score(y_test, prediction_XGB)
f1_XGB = f1_score(y_test, prediction_XGB)
AUROC_XGB = roc_auc_score(y_test, prediction_XGB)

Performance on training data using XGB Classifier: 0.9987951099094863
Performance on testing data using XGB Classifier: 0.9989420477254026


In [50]:
confusion_matrix(y_test, prediction_XGB)

array([[1321,    9],
       [   0, 7177]], dtype=int64)

In [51]:
tempResultsDf = pd.DataFrame({'Method':['XGB Classifier'], 'accuracy': [acc_XGB], 'recall': recall_XGB, 'precision': precision_XGB, 'f1 score': f1_XGB, 'AUROC': AUROC_XGB}, index={'9'})
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precision', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precision,f1 score,AUROC
1,SVM,0.974021,1.000000,0.970127,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324
3,Logistic Regression,0.992829,1.000000,0.991572,0.995768,0.977068
4,Logistic Regression KFold,0.999976,NaN,NaN,NaN,1.000000
5,AdaBoost Cassifier,0.999295,1.000000,0.999165,0.999582,0.997744
6,AdaBoost KFold,0.999248,0.940083,0.999170,0.956812,0.997667
7,Gradient Boosting Classifier,0.998942,1.000000,0.998748,0.999373,0.996617
8,Gradient Boosting KFold,0.998801,0.935125,0.998783,0.951380,0.934470
9,XGB Classifier,0.998942,1.000000,0.998748,0.999373,0.996617


In [52]:
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True, random_state = 77)
resultsXGB = cross_val_score(model_XGB,X, y, cv=kfold)
acc_KF_XGB = np.mean(abs(resultsXGB))
AUROC_KF_XGB = np.mean(abs(cross_val_score(model_XGB, X, y, scoring='roc_auc', cv=10)))
recall_KF_XGB = np.mean(abs(cross_val_score(model_XGB, X, y, scoring='recall', cv=10)))
precision_KF_XGB = np.mean(abs(cross_val_score(model_XGB, X, y, scoring='precision', cv=10)))
f1_KF_XGB = np.mean(abs(cross_val_score(model_XGB, X, y, scoring='f1', cv=10)))

In [53]:
tempResultsDf = pd.DataFrame({'Method':['XGB KFold'], 'accuracy': [acc_KF_XGB], 'recall': recall_KF_XGB, 'precision': precision_KF_XGB, 'f1 score': f1_KF_XGB, 'AUROC': AUROC_KF_XGB}, index={'10'})
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precision', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precision,f1 score,AUROC
1,SVM,0.974021,1.000000,0.970127,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324
3,Logistic Regression,0.992829,1.000000,0.991572,0.995768,0.977068
4,Logistic Regression KFold,0.999976,NaN,NaN,NaN,1.000000
5,AdaBoost Cassifier,0.999295,1.000000,0.999165,0.999582,0.997744
6,AdaBoost KFold,0.999248,0.940083,0.999170,0.956812,0.997667
7,Gradient Boosting Classifier,0.998942,1.000000,0.998748,0.999373,0.996617
8,Gradient Boosting KFold,0.998801,0.935125,0.998783,0.951380,0.934470
9,XGB Classifier,0.998942,1.000000,0.998748,0.999373,0.996617
10,XGB KFold,0.998801,0.999307,0.998588,0.998947,0.996266


In [54]:
model_CB = CatBoostClassifier(early_stopping_rounds=10, iterations = 146, learning_rate = 0.004)
model_CB.fit(X_train,y_train, eval_set=(X_test, y_test))
model_CB.score(X_test,y_test)

0:	learn: 0.6809657	test: 0.6809390	best: 0.6809390 (0)	total: 232ms	remaining: 33.6s
1:	learn: 0.6680261	test: 0.6679725	best: 0.6679725 (1)	total: 263ms	remaining: 18.9s
2:	learn: 0.6561095	test: 0.6560161	best: 0.6560161 (2)	total: 292ms	remaining: 13.9s
3:	learn: 0.6449488	test: 0.6448102	best: 0.6448102 (3)	total: 322ms	remaining: 11.4s
4:	learn: 0.6330524	test: 0.6328921	best: 0.6328921 (4)	total: 351ms	remaining: 9.91s
5:	learn: 0.6217466	test: 0.6215653	best: 0.6215653 (5)	total: 391ms	remaining: 9.12s
6:	learn: 0.6112577	test: 0.6110876	best: 0.6110876 (6)	total: 426ms	remaining: 8.47s
7:	learn: 0.5995119	test: 0.5993166	best: 0.5993166 (7)	total: 464ms	remaining: 8s
8:	learn: 0.5893222	test: 0.5890778	best: 0.5890778 (8)	total: 494ms	remaining: 7.51s
9:	learn: 0.5789427	test: 0.5786798	best: 0.5786798 (9)	total: 526ms	remaining: 7.15s
10:	learn: 0.5681676	test: 0.5678793	best: 0.5678793 (10)	total: 552ms	remaining: 6.78s
11:	learn: 0.5577252	test: 0.5574354	best: 0.5574354 (1

99:	learn: 0.1181943	test: 0.1173716	best: 0.1173716 (99)	total: 3.57s	remaining: 1.64s
100:	learn: 0.1163704	test: 0.1155457	best: 0.1155457 (100)	total: 3.61s	remaining: 1.61s
101:	learn: 0.1144241	test: 0.1136082	best: 0.1136082 (101)	total: 3.64s	remaining: 1.57s
102:	learn: 0.1125456	test: 0.1117279	best: 0.1117279 (102)	total: 3.67s	remaining: 1.53s
103:	learn: 0.1106799	test: 0.1098628	best: 0.1098628 (103)	total: 3.69s	remaining: 1.49s
104:	learn: 0.1088231	test: 0.1080051	best: 0.1080051 (104)	total: 3.74s	remaining: 1.46s
105:	learn: 0.1070890	test: 0.1062719	best: 0.1062719 (105)	total: 3.78s	remaining: 1.43s
106:	learn: 0.1052669	test: 0.1044524	best: 0.1044524 (106)	total: 3.81s	remaining: 1.39s
107:	learn: 0.1035036	test: 0.1026977	best: 0.1026977 (107)	total: 3.84s	remaining: 1.35s
108:	learn: 0.1018175	test: 0.1010133	best: 0.1010133 (108)	total: 3.87s	remaining: 1.31s
109:	learn: 0.1001229	test: 0.0993234	best: 0.0993234 (109)	total: 3.9s	remaining: 1.27s
110:	learn: 0

0.9987069472199365

In [55]:
prediction_CB = model_CB.predict(X_test)
print('Performance on training data using CatBoost:',model_CB.score(X_train,y_train))
print('Performance on testing data using CatBoost:',model_CB.score(X_test,y_test))
acc_CB = accuracy_score(y_test, prediction_CB)
recall_CB = recall_score(y_test, prediction_CB)
precision_CB = precision_score(y_test, prediction_CB)
f1_CB = f1_score(y_test, prediction_CB)
AUROC_CB = roc_auc_score(y_test, prediction_CB)

Performance on training data using CatBoost: 0.9984718467144704
Performance on testing data using CatBoost: 0.9987069472199365


In [56]:
confusion_matrix(y_test, prediction_CB)

array([[1319,   11],
       [   0, 7177]], dtype=int64)

In [57]:
tempResultsDf = pd.DataFrame({'Method':['CatBoost'], 'accuracy': [acc_CB], 'recall': recall_CB, 'precision': precision_CB, 'f1 score': f1_CB, 'AUROC': AUROC_CB}, index={'11'})
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precision', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precision,f1 score,AUROC
1,SVM,0.974021,1.000000,0.970127,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324
3,Logistic Regression,0.992829,1.000000,0.991572,0.995768,0.977068
4,Logistic Regression KFold,0.999976,NaN,NaN,NaN,1.000000
5,AdaBoost Cassifier,0.999295,1.000000,0.999165,0.999582,0.997744
6,AdaBoost KFold,0.999248,0.940083,0.999170,0.956812,0.997667
7,Gradient Boosting Classifier,0.998942,1.000000,0.998748,0.999373,0.996617
8,Gradient Boosting KFold,0.998801,0.935125,0.998783,0.951380,0.934470
9,XGB Classifier,0.998942,1.000000,0.998748,0.999373,0.996617
10,XGB KFold,0.998801,0.999307,0.998588,0.998947,0.996266


In [58]:
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True, random_state = 77)
resultsCB = cross_val_score(model_CB,X, y, cv=kfold)
acc_KF_CB = np.mean(abs(resultsCB))
AUROC_KF_CB = np.mean(abs(cross_val_score(model_CB, X, y, scoring='roc_auc', cv=10)))
recall_KF_CB = np.mean(abs(cross_val_score(model_CB, X, y, scoring='recall', cv=10)))
precision_KF_CB = np.mean(abs(cross_val_score(model_CB, X, y, scoring='precision', cv=10)))
f1_KF_CB = np.mean(abs(cross_val_score(model_CB, X, y, scoring='f1', cv=10)))

0:	learn: 0.6812454	total: 43.3ms	remaining: 6.27s
1:	learn: 0.6681418	total: 79.7ms	remaining: 5.74s
2:	learn: 0.6564962	total: 117ms	remaining: 5.59s
3:	learn: 0.6455213	total: 155ms	remaining: 5.49s
4:	learn: 0.6335053	total: 183ms	remaining: 5.17s
5:	learn: 0.6217140	total: 211ms	remaining: 4.93s
6:	learn: 0.6112883	total: 246ms	remaining: 4.88s
7:	learn: 0.5993984	total: 281ms	remaining: 4.84s
8:	learn: 0.5882143	total: 311ms	remaining: 4.74s
9:	learn: 0.5773871	total: 346ms	remaining: 4.7s
10:	learn: 0.5664619	total: 375ms	remaining: 4.6s
11:	learn: 0.5559331	total: 408ms	remaining: 4.55s
12:	learn: 0.5458367	total: 438ms	remaining: 4.48s
13:	learn: 0.5363148	total: 467ms	remaining: 4.4s
14:	learn: 0.5263595	total: 496ms	remaining: 4.33s
15:	learn: 0.5158923	total: 533ms	remaining: 4.33s
16:	learn: 0.5058254	total: 569ms	remaining: 4.32s
17:	learn: 0.4962159	total: 602ms	remaining: 4.28s
18:	learn: 0.4867347	total: 638ms	remaining: 4.27s
19:	learn: 0.4773175	total: 677ms	remainin

18:	learn: 0.4880270	total: 634ms	remaining: 4.24s
19:	learn: 0.4785940	total: 666ms	remaining: 4.2s
20:	learn: 0.4696877	total: 698ms	remaining: 4.16s
21:	learn: 0.4620397	total: 728ms	remaining: 4.1s
22:	learn: 0.4534215	total: 766ms	remaining: 4.1s
23:	learn: 0.4441777	total: 800ms	remaining: 4.07s
24:	learn: 0.4355945	total: 832ms	remaining: 4.03s
25:	learn: 0.4279233	total: 861ms	remaining: 3.97s
26:	learn: 0.4208756	total: 908ms	remaining: 4s
27:	learn: 0.4130558	total: 943ms	remaining: 3.97s
28:	learn: 0.4051750	total: 967ms	remaining: 3.9s
29:	learn: 0.3981653	total: 995ms	remaining: 3.85s
30:	learn: 0.3900607	total: 1.02s	remaining: 3.81s
31:	learn: 0.3830322	total: 1.05s	remaining: 3.76s
32:	learn: 0.3758741	total: 1.09s	remaining: 3.73s
33:	learn: 0.3695179	total: 1.12s	remaining: 3.7s
34:	learn: 0.3622500	total: 1.16s	remaining: 3.67s
35:	learn: 0.3551898	total: 1.21s	remaining: 3.69s
36:	learn: 0.3491176	total: 1.25s	remaining: 3.67s
37:	learn: 0.3428590	total: 1.28s	remai

37:	learn: 0.3465752	total: 1.23s	remaining: 3.5s
38:	learn: 0.3401699	total: 1.27s	remaining: 3.48s
39:	learn: 0.3336841	total: 1.3s	remaining: 3.45s
40:	learn: 0.3280105	total: 1.33s	remaining: 3.41s
41:	learn: 0.3214894	total: 1.36s	remaining: 3.38s
42:	learn: 0.3151460	total: 1.39s	remaining: 3.34s
43:	learn: 0.3092605	total: 1.42s	remaining: 3.3s
44:	learn: 0.3042875	total: 1.45s	remaining: 3.25s
45:	learn: 0.2993860	total: 1.48s	remaining: 3.23s
46:	learn: 0.2942236	total: 1.51s	remaining: 3.19s
47:	learn: 0.2890627	total: 1.55s	remaining: 3.16s
48:	learn: 0.2846843	total: 1.58s	remaining: 3.13s
49:	learn: 0.2795365	total: 1.62s	remaining: 3.1s
50:	learn: 0.2737432	total: 1.65s	remaining: 3.07s
51:	learn: 0.2696251	total: 1.68s	remaining: 3.04s
52:	learn: 0.2648052	total: 1.71s	remaining: 3.01s
53:	learn: 0.2600086	total: 1.74s	remaining: 2.97s
54:	learn: 0.2554946	total: 1.78s	remaining: 2.94s
55:	learn: 0.2508744	total: 1.82s	remaining: 2.93s
56:	learn: 0.2459804	total: 1.85s	r

55:	learn: 0.2515459	total: 1.8s	remaining: 2.9s
56:	learn: 0.2467051	total: 1.83s	remaining: 2.86s
57:	learn: 0.2422146	total: 1.86s	remaining: 2.83s
58:	learn: 0.2381342	total: 1.91s	remaining: 2.81s
59:	learn: 0.2339489	total: 1.94s	remaining: 2.77s
60:	learn: 0.2299992	total: 1.98s	remaining: 2.76s
61:	learn: 0.2261056	total: 2.01s	remaining: 2.72s
62:	learn: 0.2218393	total: 2.04s	remaining: 2.69s
63:	learn: 0.2178968	total: 2.07s	remaining: 2.65s
64:	learn: 0.2141042	total: 2.1s	remaining: 2.61s
65:	learn: 0.2101734	total: 2.12s	remaining: 2.57s
66:	learn: 0.2062464	total: 2.16s	remaining: 2.54s
67:	learn: 0.2024572	total: 2.19s	remaining: 2.51s
68:	learn: 0.1990367	total: 2.23s	remaining: 2.49s
69:	learn: 0.1955676	total: 2.26s	remaining: 2.45s
70:	learn: 0.1921973	total: 2.29s	remaining: 2.42s
71:	learn: 0.1887360	total: 2.31s	remaining: 2.38s
72:	learn: 0.1852573	total: 2.34s	remaining: 2.34s
73:	learn: 0.1821782	total: 2.38s	remaining: 2.31s
74:	learn: 0.1791608	total: 2.42s	

75:	learn: 0.1762620	total: 2.42s	remaining: 2.23s
76:	learn: 0.1728613	total: 2.45s	remaining: 2.19s
77:	learn: 0.1698766	total: 2.47s	remaining: 2.16s
78:	learn: 0.1666613	total: 2.5s	remaining: 2.12s
79:	learn: 0.1639924	total: 2.53s	remaining: 2.09s
80:	learn: 0.1611462	total: 2.57s	remaining: 2.06s
81:	learn: 0.1580513	total: 2.6s	remaining: 2.03s
82:	learn: 0.1550476	total: 2.64s	remaining: 2s
83:	learn: 0.1523866	total: 2.67s	remaining: 1.97s
84:	learn: 0.1497058	total: 2.71s	remaining: 1.94s
85:	learn: 0.1469135	total: 2.74s	remaining: 1.91s
86:	learn: 0.1442531	total: 2.77s	remaining: 1.88s
87:	learn: 0.1416857	total: 2.8s	remaining: 1.84s
88:	learn: 0.1395151	total: 2.83s	remaining: 1.81s
89:	learn: 0.1373066	total: 2.86s	remaining: 1.78s
90:	learn: 0.1346262	total: 2.89s	remaining: 1.75s
91:	learn: 0.1321236	total: 2.92s	remaining: 1.72s
92:	learn: 0.1298605	total: 2.95s	remaining: 1.68s
93:	learn: 0.1276297	total: 2.98s	remaining: 1.65s
94:	learn: 0.1256085	total: 3.02s	rem

...

126:	learn: 0.0749102	total: 4.15s	remaining: 621ms
127:	learn: 0.0737667	total: 4.18s	remaining: 588ms
128:	learn: 0.0726730	total: 4.21s	remaining: 555ms
129:	learn: 0.0715431	total: 4.24s	remaining: 522ms
130:	learn: 0.0704869	total: 4.27s	remaining: 489ms
131:	learn: 0.0694328	total: 4.3s	remaining: 456ms
132:	learn: 0.0683805	total: 4.33s	remaining: 424ms
133:	learn: 0.0673848	total: 4.36s	remaining: 391ms
134:	learn: 0.0664578	total: 4.39s	remaining: 358ms
135:	learn: 0.0654777	total: 4.43s	remaining: 326ms
136:	learn: 0.0644120	total: 4.46s	remaining: 293ms
137:	learn: 0.0634877	total: 4.49s	remaining: 260ms
138:	learn: 0.0625915	total: 4.52s	remaining: 228ms
139:	learn: 0.0616122	total: 4.55s	remaining: 195ms
140:	learn: 0.0607309	total: 4.58s	remaining: 162ms
141:	learn: 0.0598643	total: 4.61s	remaining: 130ms
142:	learn: 0.0590041	total: 4.64s	remaining: 97.4ms
143:	learn: 0.0581455	total: 4.68s	remaining: 65ms
144:	learn: 0.0573272	total: 4.71s	remaining: 32.5ms
145:	learn: 

0:	learn: 0.6809111	total: 33.6ms	remaining: 4.87s
1:	learn: 0.6678682	total: 70.8ms	remaining: 5.1s
2:	learn: 0.6559228	total: 101ms	remaining: 4.82s
3:	learn: 0.6446381	total: 132ms	remaining: 4.68s
4:	learn: 0.6326642	total: 160ms	remaining: 4.52s
5:	learn: 0.6212802	total: 196ms	remaining: 4.58s
6:	learn: 0.6107973	total: 229ms	remaining: 4.55s
7:	learn: 0.5990248	total: 265ms	remaining: 4.56s
8:	learn: 0.5886986	total: 294ms	remaining: 4.48s
9:	learn: 0.5782579	total: 331ms	remaining: 4.5s
10:	learn: 0.5673620	total: 360ms	remaining: 4.42s
11:	learn: 0.5568890	total: 391ms	remaining: 4.37s
12:	learn: 0.5465807	total: 436ms	remaining: 4.46s
13:	learn: 0.5369004	total: 481ms	remaining: 4.53s
14:	learn: 0.5269895	total: 513ms	remaining: 4.48s
15:	learn: 0.5165937	total: 544ms	remaining: 4.42s
16:	learn: 0.5074686	total: 575ms	remaining: 4.37s
17:	learn: 0.4978650	total: 603ms	remaining: 4.29s
18:	learn: 0.4884043	total: 640ms	remaining: 4.28s
19:	learn: 0.4789719	total: 678ms	remaini

20:	learn: 0.4723347	total: 688ms	remaining: 4.09s
21:	learn: 0.4635850	total: 717ms	remaining: 4.04s
22:	learn: 0.4550231	total: 749ms	remaining: 4.01s
23:	learn: 0.4457787	total: 783ms	remaining: 3.98s
24:	learn: 0.4371196	total: 815ms	remaining: 3.95s
25:	learn: 0.4291180	total: 850ms	remaining: 3.92s
26:	learn: 0.4206995	total: 881ms	remaining: 3.88s
27:	learn: 0.4132878	total: 913ms	remaining: 3.85s
28:	learn: 0.4057259	total: 943ms	remaining: 3.8s
29:	learn: 0.3982134	total: 976ms	remaining: 3.77s
30:	learn: 0.3919022	total: 1.01s	remaining: 3.73s
31:	learn: 0.3854727	total: 1.04s	remaining: 3.69s
32:	learn: 0.3786801	total: 1.08s	remaining: 3.69s
33:	learn: 0.3716377	total: 1.11s	remaining: 3.66s
34:	learn: 0.3648224	total: 1.15s	remaining: 3.64s
35:	learn: 0.3583512	total: 1.18s	remaining: 3.6s
36:	learn: 0.3518848	total: 1.21s	remaining: 3.56s
37:	learn: 0.3464613	total: 1.24s	remaining: 3.52s
38:	learn: 0.3401509	total: 1.27s	remaining: 3.47s
39:	learn: 0.3337087	total: 1.3s	

38:	learn: 0.3348793	total: 1.45s	remaining: 3.99s
39:	learn: 0.3284984	total: 1.49s	remaining: 3.95s
40:	learn: 0.3218778	total: 1.54s	remaining: 3.94s
41:	learn: 0.3159027	total: 1.59s	remaining: 3.93s
42:	learn: 0.3110579	total: 1.63s	remaining: 3.9s
43:	learn: 0.3054246	total: 1.67s	remaining: 3.88s
44:	learn: 0.2999646	total: 1.72s	remaining: 3.85s
45:	learn: 0.2949690	total: 1.76s	remaining: 3.83s
46:	learn: 0.2897571	total: 1.8s	remaining: 3.8s
47:	learn: 0.2845060	total: 1.84s	remaining: 3.76s
48:	learn: 0.2790535	total: 1.88s	remaining: 3.72s
49:	learn: 0.2737942	total: 1.92s	remaining: 3.68s
50:	learn: 0.2690022	total: 1.95s	remaining: 3.63s
51:	learn: 0.2644712	total: 1.99s	remaining: 3.6s
52:	learn: 0.2602821	total: 2.02s	remaining: 3.55s
53:	learn: 0.2563508	total: 2.06s	remaining: 3.5s
54:	learn: 0.2521665	total: 2.1s	remaining: 3.47s
55:	learn: 0.2483874	total: 2.14s	remaining: 3.43s
56:	learn: 0.2435645	total: 2.17s	remaining: 3.4s
57:	learn: 0.2391221	total: 2.22s	rema

53:	learn: 0.2586099	total: 2.22s	remaining: 3.79s
54:	learn: 0.2548699	total: 2.25s	remaining: 3.73s
55:	learn: 0.2501959	total: 2.29s	remaining: 3.67s
56:	learn: 0.2456706	total: 2.32s	remaining: 3.62s
57:	learn: 0.2410181	total: 2.35s	remaining: 3.57s
58:	learn: 0.2364859	total: 2.38s	remaining: 3.52s
59:	learn: 0.2325359	total: 2.42s	remaining: 3.46s
60:	learn: 0.2281289	total: 2.45s	remaining: 3.41s
61:	learn: 0.2239059	total: 2.48s	remaining: 3.36s
62:	learn: 0.2196808	total: 2.51s	remaining: 3.31s
63:	learn: 0.2156779	total: 2.54s	remaining: 3.25s
64:	learn: 0.2119643	total: 2.6s	remaining: 3.24s
65:	learn: 0.2081950	total: 2.64s	remaining: 3.19s
66:	learn: 0.2044363	total: 2.68s	remaining: 3.16s
67:	learn: 0.2008033	total: 2.75s	remaining: 3.16s
68:	learn: 0.1972858	total: 2.8s	remaining: 3.13s
69:	learn: 0.1933886	total: 2.86s	remaining: 3.1s
70:	learn: 0.1898445	total: 2.91s	remaining: 3.08s
71:	learn: 0.1860944	total: 2.96s	remaining: 3.04s
72:	learn: 0.1826151	total: 3s	rem

In [59]:
tempResultsDf = pd.DataFrame({'Method':['CatBoost KFold'], 'accuracy': [acc_KF_CB], 'recall': recall_KF_CB, 'precision': precision_KF_CB, 'f1 score': f1_KF_CB, 'AUROC': AUROC_KF_CB}, index={'12'})
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precision', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precision,f1 score,AUROC
1,SVM,0.974021,1.000000,0.970127,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324
3,Logistic Regression,0.992829,1.000000,0.991572,0.995768,0.977068
4,Logistic Regression KFold,0.999976,NaN,NaN,NaN,1.000000
5,AdaBoost Cassifier,0.999295,1.000000,0.999165,0.999582,0.997744
6,AdaBoost KFold,0.999248,0.940083,0.999170,0.956812,0.997667
7,Gradient Boosting Classifier,0.998942,1.000000,0.998748,0.999373,0.996617
8,Gradient Boosting KFold,0.998801,0.935125,0.998783,0.951380,0.934470
9,XGB Classifier,0.998942,1.000000,0.998748,0.999373,0.996617
10,XGB KFold,0.998801,0.999307,0.998588,0.998947,0.996266


In [60]:
model_KN = KNeighborsClassifier(n_neighbors=10)
model_KN.fit(X_train,y_train)
model_KN.score(X_test, y_test)

0.8859762548489479

In [61]:
prediction_KN = model_KN.predict(X_test)
print('Performance on training data using KNeighbors:',model_KN.score(X_train,y_train))
print('Performance on testing data using KNeighbors:',model_KN.score(X_test,y_test))
acc_KN = accuracy_score(y_test, prediction_KN)
recall_KN = recall_score(y_test, prediction_KN)
precision_KN = precision_score(y_test, prediction_KN)
f1_KN = f1_score(y_test, prediction_KN)
AUROC_KN = roc_auc_score(y_test, prediction_KN)

Performance on training data using KNeighbors: 0.9033736922534383
Performance on testing data using KNeighbors: 0.8859762548489479


In [62]:
tempResultsDf = pd.DataFrame({'Method':['KNeighbors'], 'accuracy': [acc_KN], 'recall': recall_KN, 'precision': precision_KN, 'f1 score': f1_KN, 'AUROC': AUROC_KN}, index={'13'})
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precision', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precision,f1 score,AUROC
1,SVM,0.974021,1.000000,0.970127,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324
3,Logistic Regression,0.992829,1.000000,0.991572,0.995768,0.977068
4,Logistic Regression KFold,0.999976,NaN,NaN,NaN,1.000000
5,AdaBoost Cassifier,0.999295,1.000000,0.999165,0.999582,0.997744
6,AdaBoost KFold,0.999248,0.940083,0.999170,0.956812,0.997667
7,Gradient Boosting Classifier,0.998942,1.000000,0.998748,0.999373,0.996617
8,Gradient Boosting KFold,0.998801,0.935125,0.998783,0.951380,0.934470
9,XGB Classifier,0.998942,1.000000,0.998748,0.999373,0.996617
10,XGB KFold,0.998801,0.999307,0.998588,0.998947,0.996266


In [63]:
confusion_matrix(y_test, prediction_KN)

array([[ 442,  888],
       [  82, 7095]], dtype=int64)

In [64]:
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True, random_state = 77)
resultsKN = cross_val_score(model_KN, X, y, cv=kfold)
acc_KF_KN = np.mean(abs(resultsKN))
AUROC_KF_KN = np.mean(abs(cross_val_score(model_KN, X, y, scoring='roc_auc', cv=10)))
recall_KF_KN = np.mean(abs(cross_val_score(model_KN, X, y, scoring='recall', cv=10)))
precision_KF_KN = np.mean(abs(cross_val_score(model_KN, X, y, scoring='precision', cv=10)))
f1_KF_KN = np.mean(abs(cross_val_score(model_KN, X, y, scoring='f1', cv=10)))

In [65]:
tempResultsDf = pd.DataFrame({'Method':['KNeighbours KFold'], 'accuracy': [acc_KF_KN], 'recall': recall_KF_KN, 'precision': precision_KF_KN, 'f1 score': f1_KF_KN, 'AUROC': AUROC_KF_KN}, index={'14'})
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precision', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precision,f1 score,AUROC
1,SVM,0.974021,1.000000,0.970127,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324
3,Logistic Regression,0.992829,1.000000,0.991572,0.995768,0.977068
4,Logistic Regression KFold,0.999976,NaN,NaN,NaN,1.000000
5,AdaBoost Cassifier,0.999295,1.000000,0.999165,0.999582,0.997744
6,AdaBoost KFold,0.999248,0.940083,0.999170,0.956812,0.997667
7,Gradient Boosting Classifier,0.998942,1.000000,0.998748,0.999373,0.996617
8,Gradient Boosting KFold,0.998801,0.935125,0.998783,0.951380,0.934470
9,XGB Classifier,0.998942,1.000000,0.998748,0.999373,0.996617
10,XGB KFold,0.998801,0.999307,0.998588,0.998947,0.996266


In [66]:
model_RF = RandomForestClassifier(n_estimators=20)
model_RF.fit(X_train, y_train)
model_RF.score(X_test, y_test)

0.9991771482308687

In [67]:
prediction_RF = model_RF.predict(X_test)
print('Performance on training data using RandomForest Classifier:',model_RF.score(X_train,y_train))
print('Performance on testing data using RandomForest Classifier:',model_RF.score(X_test,y_test))
acc_RF = accuracy_score(y_test, prediction_RF)
recall_RF = recall_score(y_test, prediction_RF)
precision_RF = precision_score(y_test, prediction_RF)
f1_RF = f1_score(y_test, prediction_RF)
AUROC_RF = roc_auc_score(y_test, prediction_RF)

Performance on training data using RandomForest Classifier: 0.9999412248736335
Performance on testing data using RandomForest Classifier: 0.9991771482308687


In [68]:
tempResultsDf = pd.DataFrame({'Method':['RandomForest'], 'accuracy': [acc_RF], 'recall': recall_RF, 'precission': precision_RF, 'f1 score': f1_RF, 'AUROC': AUROC_RF}, index={'15'})
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precission', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precission,f1 score,AUROC
1,SVM,0.974021,1.000000,NaN,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324
3,Logistic Regression,0.992829,1.000000,NaN,0.995768,0.977068
4,Logistic Regression KFold,0.999976,NaN,NaN,NaN,1.000000
5,AdaBoost Cassifier,0.999295,1.000000,NaN,0.999582,0.997744
6,AdaBoost KFold,0.999248,0.940083,NaN,0.956812,0.997667
7,Gradient Boosting Classifier,0.998942,1.000000,NaN,0.999373,0.996617
8,Gradient Boosting KFold,0.998801,0.935125,NaN,0.951380,0.934470
9,XGB Classifier,0.998942,1.000000,NaN,0.999373,0.996617
10,XGB KFold,0.998801,0.999307,NaN,0.998947,0.996266


In [69]:
confusion_matrix(y_test, prediction_RF)

array([[1323,    7],
       [   0, 7177]], dtype=int64)

In [70]:
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True, random_state = 77)
resultsRF = cross_val_score(model_RF,X, y, cv=kfold)
acc_KF_RF = np.mean(abs(resultsRF))
AUROC_KF_RF = np.mean(abs(cross_val_score(model_RF, X, y, scoring='roc_auc', cv=10)))
recall_KF_RF = np.mean(abs(cross_val_score(model_RF, X, y, scoring='recall', cv=10)))
precision_KF_RF = np.mean(abs(cross_val_score(model_RF, X, y, scoring='precision', cv=10)))
f1_KF_RF = np.mean(abs(cross_val_score(model_RF, X, y, scoring='f1', cv=10)))

In [71]:
tempResultsDf = pd.DataFrame({'Method':['RandomForest KFold'], 'accuracy': [acc_KF_RF], 'recall': recall_KF_KN, 'precision': precision_KF_RF, 'f1 score': f1_KF_RF, 'AUROC': AUROC_KF_RF}, index={'16'})
results = pd.concat([results, tempResultsDf])
results = results[['Method', 'accuracy', 'recall', 'precission', 'f1 score', 'AUROC']]
results

,Method,accuracy,recall,precission,f1 score,AUROC
1,SVM,0.974021,1.000000,NaN,0.984837,0.916917
2,SVC K Fold,0.970847,NaN,NaN,NaN,0.999324
3,Logistic Regression,0.992829,1.000000,NaN,0.995768,0.977068
4,Logistic Regression KFold,0.999976,NaN,NaN,NaN,1.000000
5,AdaBoost Cassifier,0.999295,1.000000,NaN,0.999582,0.997744
6,AdaBoost KFold,0.999248,0.940083,NaN,0.956812,0.997667
7,Gradient Boosting Classifier,0.998942,1.000000,NaN,0.999373,0.996617
8,Gradient Boosting KFold,0.998801,0.935125,NaN,0.951380,0.934470
9,XGB Classifier,0.998942,1.000000,NaN,0.999373,0.996617
10,XGB KFold,0.998801,0.999307,NaN,0.998947,0.996266


In [72]:
results.sort_values('AUROC')

,Method,accuracy,recall,precission,f1 score,AUROC
13,KNeighbors,0.885976,0.988575,NaN,0.936016,0.660453
1,SVM,0.974021,1.000000,NaN,0.984837,0.916917
8,Gradient Boosting KFold,0.998801,0.935125,NaN,0.951380,0.934470
14,KNeighbours KFold,0.950017,0.998504,NaN,0.971329,0.944718
3,Logistic Regression,0.992829,1.000000,NaN,0.995768,0.977068
11,CatBoost,0.998707,1.000000,NaN,0.999234,0.995865
10,XGB KFold,0.998801,0.999307,NaN,0.998947,0.996266
7,Gradient Boosting Classifier,0.998942,1.000000,NaN,0.999373,0.996617
9,XGB Classifier,0.998942,1.000000,NaN,0.999373,0.996617
15,RandomForest,0.999177,1.000000,0.999026,0.999513,0.997368


As we can observe the best performace occurs among KFold validated models. The highest efficiency on AUROC score has Logistic Regression with KFold validation, next is CatBoost with KFold and then there go SVM and Random Forest Classifier with KFold. The fifth is first model train without validation - AdaBooost Classifiers. Those 5 models I'm going to check with PCA method